In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
# https://theleftjoin.com/how-to-write-a-pandas-dataframe-to-an-sqlite-table/

In [3]:
# import csv
# df.id = df.id.astype(str)
# df.to_csv('test.txt', index=False, quotechar='"',header=None, quoting=csv.QUOTE_NONNUMERIC)

### ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

In [4]:
dep_df_temp = pd.read_excel('datos_diana/Indicadores2021_departamentos.xlsx')
mun_df_temp = pd.read_excel('datos_diana/Indicadores2021_municipios.xlsx')
man_df_temp = pd.read_csv('datos_diana/Indicadores2021_manzanas.csv', 
                            usecols = ['DIVIPOLA', 'Valor', 'Dimensión', 'Nombre_tablero'])

## 01_Departamentos.csv

In [5]:
dep_df = dep_df_temp.copy()
dep_df = dep_df[['DIVIPOLA', 'DEPARTAMENTO']]
dep_df['DIVIPOLA'] = dep_df['DIVIPOLA'].astype(str).str.zfill(2)
dep_df['DEPARTAMENTO'] = dep_df['DEPARTAMENTO'].str.strip().str.capitalize()
dep_df = dep_df.drop_duplicates();

dep_df.reset_index(drop=True, inplace=True)
dep_df.reset_index(inplace=True)
dep_df['index'] = dep_df['index'] + 1

dep_df = dep_df[['index', 'DIVIPOLA', 'DEPARTAMENTO']]
dep_df.rename(columns = {'index':'id', 'DIVIPOLA':'divipola', 'DEPARTAMENTO':'nombre'}, inplace = True)

# dep_df.to_excel('./01_Departamentos.xlsx', index=False)
dep_df.to_csv('./01_Departamentos.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
dep_df.head(10)

,id,divipola,nombre
0,1,05,Antioquia
1,2,08,Atlántico
2,3,11,Bogotá d.c
3,4,13,Bolívar
4,5,15,Boyacá
5,6,17,Caldas
6,7,18,Caquetá
7,8,19,Cauca
8,9,20,Cesar
9,10,23,Córdoba


## 02_Municipios.csv

In [6]:
mun_df = mun_df_temp.copy()
mun_df = mun_df[['DIVIPOLA', 'MUNICIPIO']]
mun_df['DIVIPOLA'] = mun_df['DIVIPOLA'].astype(str).str.zfill(5)
mun_df['MUNICIPIO'] = mun_df['MUNICIPIO'].str.strip().str.capitalize()
mun_df = mun_df.drop_duplicates();

mun_df["dep_divipola"] = mun_df['DIVIPOLA'].str[:2]
mun_df = mun_df.merge(dep_df[['id', 'divipola']], left_on='dep_divipola', right_on='divipola', how='left')

mun_df.reset_index(drop=True, inplace=True)
mun_df.reset_index(inplace=True)
mun_df['index'] = mun_df['index'] + 1

mun_df = mun_df[['index', 'DIVIPOLA', 'MUNICIPIO', 'id',]]
mun_df.rename(columns = {'index':'id', 'id':'departamento_id', 'DIVIPOLA':'divipola', 'MUNICIPIO':'nombre'}, inplace = True)
mun_df['departamento'] = ''

# mun_df.to_excel('./02_Municipios.xlsx', index=False)
mun_df.to_csv('./02_Municipios.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
mun_df.head(10)

,id,divipola,nombre,departamento_id,departamento
0,1,05001,Medellín,1,
1,2,05002,Abejorral,1,
2,3,05004,Abriaquí,1,
3,4,05021,Alejandría,1,
4,5,05030,Amagá,1,
5,6,05031,Amalfi,1,
6,7,05034,Andes,1,
7,8,05036,Angelópolis,1,
8,9,05038,Angostura,1,
9,10,05040,Anorí,1,


## 03_Manzanas.csv

In [7]:
man_df = man_df_temp.copy()
man_df = man_df[['DIVIPOLA']]
man_df['DIVIPOLA'] = man_df['DIVIPOLA'].astype(str).str.zfill(22)
# man_df['MUNICIPIO'] = man_df['MUNICIPIO'].str.strip().str.capitalize()
man_df = man_df.drop_duplicates();

man_df["mun_divipola"] = man_df['DIVIPOLA'].str[:5]
man_df = man_df.merge(mun_df[['id', 'divipola']], left_on='mun_divipola', right_on='divipola', how='left')

man_df.reset_index(drop=True, inplace=True)
man_df.reset_index(inplace=True)
man_df['index'] = man_df['index'] + 1

man_df = man_df[['index', 'DIVIPOLA', 'id',]]
man_df.rename(columns = {'index':'id', 'id':'municipio_id', 'DIVIPOLA':'divipola'}, inplace = True)
man_df['municipio'] = ''

# man_df.to_excel('./03_Manzanas.xlsx', index=False)
man_df.to_csv('./03_Manzanas.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
man_df.head(10)

,id,divipola,municipio_id,municipio
0,1,0500210000000000010101,2,
1,2,0500210000000000010102,2,
2,3,0500210000000000010103,2,
3,4,0500210000000000010104,2,
4,5,0500210000000000010105,2,
5,6,0500210000000000010106,2,
6,7,0500210000000000010107,2,
7,8,0500210000000000010108,2,
8,9,0500210000000000010109,2,
9,10,0500210000000000010110,2,


## 04_Dimensiones.csv

In [8]:
dim_df = pd.concat([dep_df_temp['Dimensión'], mun_df_temp['Dimensión'], mun_df_temp['Dimensión']]).to_frame()
dim_df.rename(columns = {'Dimensión':'nombre'}, inplace = True)
dim_df['nombre'] = dim_df['nombre'].replace('Ind_Pobreza_Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
dim_df['nombre'] = dim_df['nombre'].replace('Indice de Pobreza Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
dim_df['nombre'] = dim_df['nombre'].str.strip().str.capitalize()
dim_df = dim_df.drop_duplicates();
dim_df.sort_values(by=['nombre'], inplace=True)

dim_df.reset_index(drop=True, inplace=True)
dim_df.reset_index(inplace=True)
dim_df['index'] = dim_df['index'] + 1

dim_df.rename(columns = {'index':'id'}, inplace = True)
# dim_df.to_excel('./04_Dimensiones.xlsx', index=False)
dim_df.to_csv('./04_Dimensiones.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
dim_df.head(100)

,id,nombre
0,1,Accesibilidad
1,2,Calidad
2,3,Disponibilidad
3,4,Indice de pobreza multidimensional y tasas mor...
4,5,Población
5,6,Utilización


## 05_Indicadores.csv

In [9]:
def es_porcentual(indicador):
    if any(substring in indicador.lower() for substring in ['%', 'tasa']):
        return 1
    return 0

ind_df = pd.concat([
    dep_df_temp[['Dimensión', 'Nombre_tablero']], 
    mun_df_temp[['Dimensión', 'Nombre_tablero']],
    man_df_temp[['Dimensión', 'Nombre_tablero']]
])
ind_df.rename(columns = {'Dimensión':'dimension', 'Nombre_tablero':'indicador'}, inplace = True)
ind_df['dimension'] = ind_df['dimension'].replace('Ind_Pobreza_Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
ind_df['dimension'] = ind_df['dimension'].replace('Indice de Pobreza Multidimensional','Indice de Pobreza Multidimensional y tasas mortalidad')
ind_df['dimension'] = ind_df['dimension'].str.strip().str.capitalize()
ind_df['indicador'] = ind_df['indicador'].replace("% \'", "(%) ", regex=True)
ind_df['indicador'] = ind_df['indicador'].str.strip().str.capitalize()
ind_df = ind_df.drop_duplicates();
ind_df["es_porcentual"] = ind_df["indicador"].apply(es_porcentual)
ind_df.sort_values(by=['indicador'], inplace=True)

ind_df = ind_df.merge(dim_df[['nombre', 'id']], left_on='dimension', right_on='nombre', how='left')

ind_df.reset_index(drop=True, inplace=True)
ind_df.reset_index(inplace=True)
ind_df['index'] = ind_df['index'] + 1

ind_df = ind_df[['index', 'indicador', 'id', 'es_porcentual']]
ind_df.rename(columns = {'id':'dimension_id', 'index':'id', 'indicador':'nombre'}, inplace = True)
# ind_df.to_excel('./05_Indicadores.xlsx', index=False)
ind_df.to_csv('./05_Indicadores.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
ind_df.head()

,id,nombre,dimension_id,es_porcentual
0,1,(%) mujeres en edad reproductiva sin cobertura...,1,1
1,2,(%) mujeres en edad reproductiva sin cobertura...,1,1
2,3,(%) mujeres rurales sin cobertura en en edad r...,1,1
3,4,(%) mujeres urbanas sin cobertura en en edad r...,1,1
4,5,(%) población infantil rural sin cobertura,1,1


## 06_RegistroIndiceDepartamento.csv

In [10]:
reg_dep = dep_df_temp.copy()
reg_dep = reg_dep[['DIVIPOLA', 'Nombre_tablero', 'Valor']]

reg_dep['DIVIPOLA'] = reg_dep['DIVIPOLA'].astype(str).str.zfill(2)
reg_dep = reg_dep.merge(dep_df[['id', 'divipola']], left_on='DIVIPOLA', right_on='divipola', how='left')
reg_dep.drop(['DIVIPOLA', 'divipola'], axis=1, inplace=True)
reg_dep.rename(columns = {'id':'departamento_id'}, inplace = True)

reg_dep['Nombre_tablero'] = reg_dep['Nombre_tablero'].replace("% \'", "(%) ", regex=True)
reg_dep['Nombre_tablero'] = reg_dep['Nombre_tablero'].str.strip().str.capitalize()
reg_dep = reg_dep.merge(ind_df[['id', 'nombre']], left_on='Nombre_tablero', right_on='nombre', how='left')
reg_dep.drop(['nombre', 'Nombre_tablero'], axis=1, inplace=True)
reg_dep.rename(columns = {'id':'indicador_id'}, inplace = True)

reg_dep.reset_index(drop=True, inplace=True)
reg_dep.reset_index(inplace=True)
reg_dep['index'] = reg_dep['index'] + 1

reg_dep = reg_dep[['index', 'Valor', 'indicador_id', 'departamento_id']]
reg_dep.rename(columns = {'index':'id', 'Valor':'valor'}, inplace = True)
# reg_dep.to_excel('./06_RegistroIndiceDepartamento.xlsx', index=False)
reg_dep.to_csv('./06_RegistroIndiceDepartamento.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
reg_dep.head(10)

,id,valor,indicador_id,departamento_id
0,1,17.1,34,1
1,2,20.1,34,2
2,3,4.4,34,3
3,4,32.4,34,4
4,5,16.6,34,5
5,6,15.3,34,6
6,7,28.7,34,7
7,8,28.7,34,8
8,9,33.2,34,9
9,10,36.7,34,10


## 07_RegistroIndiceMunicipio.csv

In [11]:
reg_mun = mun_df_temp.copy()
reg_mun = reg_mun[['DIVIPOLA', 'Nombre_tablero', 'Valor']]

reg_mun['DIVIPOLA'] = reg_mun['DIVIPOLA'].astype(str).str.zfill(5)
reg_mun = reg_mun.merge(mun_df[['id', 'divipola']], left_on='DIVIPOLA', right_on='divipola', how='left')
reg_mun.drop(['DIVIPOLA', 'divipola'], axis=1, inplace=True)
reg_mun.rename(columns = {'id':'municipio_id'}, inplace = True)

reg_mun['Nombre_tablero'] = reg_mun['Nombre_tablero'].replace("% \'", "(%) ", regex=True)
reg_mun['Nombre_tablero'] = reg_mun['Nombre_tablero'].str.strip().str.capitalize()
reg_mun = reg_mun.merge(ind_df[['id', 'nombre']], left_on='Nombre_tablero', right_on='nombre', how='left')
reg_mun.drop(['nombre', 'Nombre_tablero'], axis=1, inplace=True)
reg_mun.rename(columns = {'id':'indicador_id'}, inplace = True)

reg_mun.reset_index(drop=True, inplace=True)
reg_mun.reset_index(inplace=True)
reg_mun['index'] = reg_mun['index'] + 1

reg_mun = reg_mun[['index', 'Valor', 'indicador_id', 'municipio_id']]
reg_mun.rename(columns = {'index':'id', 'Valor':'valor'}, inplace = True)
# reg_mun.to_excel('./07_RegistroIndiceMunicipio.xlsx', index=False)
reg_mun.to_csv('./07_RegistroIndiceMunicipio.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
reg_mun.head(10)

,id,valor,indicador_id,municipio_id
0,1,12.8,34,1
1,2,40.6,34,2
2,3,34.8,34,3
3,4,31.9,34,4
4,5,22.9,34,5
5,6,43.3,34,6
6,7,40.1,34,7
7,8,30.7,34,8
8,9,53.8,34,9
9,10,53.3,34,10


## 08_RegistroIndiceManzana.csv

In [12]:
reg_man = man_df_temp.copy()
reg_man = reg_man[['DIVIPOLA', 'Nombre_tablero', 'Valor']]

reg_man['DIVIPOLA'] = reg_man['DIVIPOLA'].astype(str).str.zfill(22)
reg_man = reg_man.merge(man_df[['id', 'divipola']], left_on='DIVIPOLA', right_on='divipola', how='left')
reg_man.drop(['DIVIPOLA', 'divipola'], axis=1, inplace=True)
reg_man.rename(columns = {'id':'manzana_id'}, inplace = True)

reg_man['Nombre_tablero'] = reg_man['Nombre_tablero'].replace("% \'", "(%) ", regex=True)
reg_man['Nombre_tablero'] = reg_man['Nombre_tablero'].str.strip().str.capitalize()
reg_man = reg_man.merge(ind_df[['id', 'nombre']], left_on='Nombre_tablero', right_on='nombre', how='left')
reg_man.drop(['nombre', 'Nombre_tablero'], axis=1, inplace=True)
reg_man.rename(columns = {'id':'indicador_id'}, inplace = True)

reg_man.reset_index(drop=True, inplace=True)
reg_man.reset_index(inplace=True)
reg_man['index'] = reg_man['index'] + 1

reg_man = reg_man[['index', 'Valor', 'indicador_id', 'manzana_id']]
reg_man.rename(columns = {'index':'id', 'Valor':'valor'}, inplace = True)
# reg_man.to_excel('./08_RegistroIndiceManzana.xlsx', index=False)
reg_man.to_csv('./08_RegistroIndiceManzana.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
reg_man.head(10)

,id,valor,indicador_id,manzana_id
0,1,57.0,92,1
1,2,57.0,93,1
2,3,102.0,92,2
3,4,102.0,93,2
4,5,82.0,92,3
5,6,82.0,93,3
6,7,74.0,92,4
7,8,74.0,93,4
8,9,0.0,92,5
9,10,0.0,93,5
